In [1]:

%load_ext bigdata

In [2]:
%hive_start

In [3]:

%timeout 300

In [4]:
!hdfs dfs -ls /tmp

Found 5 items
drwxrwx---   - root supergroup          0 2020-01-20 22:44 /tmp/hadoop-yarn
drwx-wx-wx   - root supergroup          0 2020-01-20 23:11 /tmp/hive
drwxrwx---   - root supergroup          0 2020-01-21 01:02 /tmp/hivep4
drwxr-xr-x   - root supergroup          0 2020-01-21 01:09 /tmp/hivep6
drwxrwx---   - root supergroup          0 2020-01-21 00:57 /tmp/output


In [5]:
!hdfs dfs -mkdir /tmp/hivep2

In [6]:
!hdfs dfs -ls /tmp/

Found 6 items
drwxrwx---   - root supergroup          0 2020-01-20 22:44 /tmp/hadoop-yarn
drwx-wx-wx   - root supergroup          0 2020-01-20 23:11 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-01-21 01:13 /tmp/hivep2
drwxrwx---   - root supergroup          0 2020-01-21 01:02 /tmp/hivep4
drwxr-xr-x   - root supergroup          0 2020-01-21 01:09 /tmp/hivep6
drwxrwx---   - root supergroup          0 2020-01-21 00:57 /tmp/output


In [8]:
!hdfs dfs -copyFromLocal data.tsv /tmp/hivep2/

In [9]:
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/hivep2

Found 1 items
-rw-r--r--   1 root supergroup        677 2020-01-21 01:13 /tmp/hivep2/data.tsv


In [ ]:
%%hive
DROP TABLE IF EXISTS data;

CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)

ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';

FIELDS TERMINATED BY '\t';
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [26]:
%%hive
SHOW CREATE TABLE data;

SHOW CREATE TABLE data;
OK
CREATE TABLE `data`(
  `letra` string, 
  `fecha` string, 
  `numero` int)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
WITH SERDEPROPERTIES ( 
  'field.delim'='\t', 
  'serialization.format'='\t') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  'hdfs://0.0.0.0:9000/user/hive/warehouse/data'
TBLPROPERTIES (
  'transient_lastDdlTime'='1579477924')
Time taken: 0.153 seconds, Fetched: 17 row(s)


In [27]:
%%hive
LOAD DATA INPATH '/tmp/hivep2/data.tsv' OVERWRITE
INTO TABLE data;

LOAD DATA INPATH '/tmp/hivep2/data.tsv' OVERWRITE
INTO TABLE data;
Loading data to table default.data
OK
Time taken: 1.024 seconds


In [28]:
%%hive
SHOW TBLPROPERTIES data;

SHOW TBLPROPERTIES data;
OK
numFiles	1
numRows	0
rawDataSize	0
totalSize	677
transient_lastDdlTime	1579477948
Time taken: 0.031 seconds, Fetched: 5 row(s)


In [29]:
%%hive
SELECT * FROM data;

SELECT * FROM data;
OK
B	1999-08-28	14
E	1999-12-06	12
E	1993-07-21	17
C	1991-02-12	13
E	1995-04-25	16
A	1992-08-22	14
B	1999-06-11	12
E	1993-01-27	13
E	1999-09-10	11
E	1990-05-03	16
E	1994-02-14	5
A	1988-04-27	12
A	1990-10-06	10
E	1985-02-12	16
E	1998-09-14	16
B	1994-08-30	17
A	1997-12-15	13
B	1995-08-23	10
B	1998-11-22	13
B	1997-04-09	14
E	1993-12-27	18
E	1999-01-14	15
A	1992-09-19	18
B	1993-03-02	14
B	1999-10-21	13
A	1990-08-31	12
C	1994-01-25	6
E	1990-02-09	18
A	1990-09-26	14
A	1993-05-08	16
B	1995-09-06	14
E	1991-02-18	14
A	1993-01-11	14
A	1990-07-22	18
C	1994-09-09	15
C	1994-07-27	7
D	1990-10-10	15
A	1990-09-05	11
B	1991-10-01	15
A	1994-10-25	13
Time taken: 1.492 seconds, Fetched: 40 row(s)


In [31]:
%%hive

CREATE TABLE word_counts
AS 
SELECT letra, numero 
    FROM data
GROUP BY
    letra, numero
;


INSERT OVERWRITE DIRECTORY '/tmp/hivep2'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM word_counts;

CREATE TABLE word_counts
AS 
SELECT letra, numero 
    FROM data
GROUP BY
    letra, numero
;
Query ID = root_20200119235648_0723b207-816c-4050-95db-7b41454498d8
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579476048284_0001, Tracking URL = http://219129cd0edb:8088/proxy/application_1579476048284_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579476048284_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-19 23:57:01,346 Stage-1 map = 0%,  reduce = 0%
2020-01-19 23:57:06,879 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.82 sec
2020-01-19 23:57:11,272 Stag

In [32]:
## Ejecucion
!hive -S -e 'source hivep2.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: hivep2.hql is not a file.


In [34]:

## Se lista el contenido del archivo.
!hdfs dfs -ls /tmp/hivep2

Found 1 items
-rwxr-xr-x   1 root supergroup        132 2020-01-19 23:57 /tmp/hivep2/000000_0


In [36]:
!hdfs dfs -cat /tmp/hivep2/000000_0 

A,10
A,11
A,12
A,13
A,14
A,16
A,18
B,10
B,12
B,13
B,14
B,15
B,17
C,6
C,7
C,13
C,15
D,15
E,5
E,11
E,12
E,13
E,14
E,15
E,16
E,17
E,18


In [37]:
!hdfs dfs -cat /tmp/hivep2/000000_0 

A,10
A,11
A,12
A,13
A,14
A,16
A,18
B,10
B,12
B,13
B,14
B,15
B,17
C,6
C,7
C,13
C,15
D,15
E,5
E,11
E,12
E,13
E,14
E,15
E,16
E,17
E,18


In [40]:
!hdfs dfs -copyToLocal /tmp/hivep2/000000_0 output;